# 전처리 한 ver -> 망함
## 프로젝트 주제 : 성경 번역 프로그램 

In [1]:
import pymysql

# 데이터 베이스 연결하기
conn = pymysql.connect(host='1.251.203.204',
                       user='root',
                       password='kdt5',
                       db='Team4',
                       charset='utf8',
                       port=33065)

curs = conn.cursor()

# 검색 명령어 사용 
sql = "SELECT eng.text as eng_text, kor.text as kor_text FROM language_en eng join language_ko kor on eng.id = kor.id;"
curs.execute(sql)
result = curs.fetchall()
print("현재 테이블의 데이터수는 총 {}개 입니다.".format(len(result)))

현재 테이블의 데이터수는 총 31102개 입니다.


In [2]:
import pandas as pd
df = pd.read_sql(sql, conn)

# 데이터베이스 연결 종료
conn.close()


C:\Users\kdp\AppData\Local\Temp\ipykernel_11444\1632777522.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


In [14]:
# 쿼리 -> dataframe결과 출력
df.head()

,eng_text,kor_text
0,In the beginning God created the heaven and th...,태초에 하나님이 천지를 창조하시니라
1,"And the earth was without form, and void; and ...",땅이 혼돈하고 공허하며 흑암이 깊음 위에 있고 하나님의 신은 수면에 운행하시니라
2,"And God said, Let there be light: and there wa...",하나님이 가라사대 빛이 있으라 하시매 빛이 있었고
3,"And God saw the light, that it was good: and G...",그 빛이 하나님의 보시기에 좋았더라 하나님이 빛과 어두움을 나누사
4,"And God called the light Day, and the darkness...",빛을 낮이라 칭하시고 어두움을 밤이라 칭하시니라 저녁이 되며 아침이 되니 이는 첫째...


In [18]:
import numpy as np
import re

def convert_to_lowercase(eng_str):
    
    # df.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)
    return eng_str.lower()


# def remove_special_characters(text):

#     # 정규 표현식을 사용하여 특수문자 제거
#     cleaned_text = re.sub(r'[^\w\s]', '', text)
#     return cleaned_text

def preprocess_sentence(sent):

  # 단어와 구두점 사이에 공백을 만듭니다.
  # Ex) "he is a boy." => "he is a boy ."
  sent = re.sub(r"([?.!,:;])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
  sent = re.sub(r"[^a-zA-Z가-힣!.?:;]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent

In [19]:
df["eng_text"] = df["eng_text"].apply(convert_to_lowercase)
df["eng_text"] = df["eng_text"].apply(preprocess_sentence)
df["kor_text"] = df["kor_text"].apply(preprocess_sentence)

In [20]:
# 전처리 후 결과 출력 
df.head()

,eng_text,kor_text
0,in the beginning god created the heaven and th...,태초에 하나님이 천지를 창조하시니라
1,and the earth was without form and void ; and ...,땅이 혼돈하고 공허하며 흑암이 깊음 위에 있고 하나님의 신은 수면에 운행하시니라
2,and god said let there be light : and there wa...,하나님이 가라사대 빛이 있으라 하시매 빛이 있었고
3,and god saw the light that it was good : and g...,그 빛이 하나님의 보시기에 좋았더라 하나님이 빛과 어두움을 나누사
4,and god called the light day and the darkness ...,빛을 낮이라 칭하시고 어두움을 밤이라 칭하시니라 저녁이 되며 아침이 되니 이는 첫째...


### 데이터 프레임 iter로 만들기(내가 쓰는 version은 TabularDataset지원 X)

In [21]:
def dataframe_iterator(df):
    for i, row in df.iterrows():
        yield row["eng_text"], row["kor_text"] 

split_idx = int(len(df) * 0.8)  # 전체 데이터 중 80%를 훈련 데이터셋으로 사용
train_df = df.iloc[:] # 걍 다 쓰세욤
valid_df = df.iloc[split_idx:]

# train_iter = dataframe_iterator(train_df)
# valid_iter = dataframe_iterator(valid_df)

In [22]:
train_df["eng_text"]

0        in the beginning god created the heaven and th...
1        and the earth was without form and void ; and ...
2        and god said let there be light : and there wa...
3        and god saw the light that it was good : and g...
4        and god called the light day and the darkness ...
                               ...                        
31097    and the spirit and the bride say come . and le...
31098    for i testify unto every man that heareth the ...
31099    and if any man shall take away from the words ...
31100    he which testifieth these things saith surely ...
31101    the grace of our lord jesus christ be with you...
Name: eng_text, Length: 31102, dtype: object

In [23]:
train_df["eng_text"].shape[0], valid_df.shape[0]

(31102, 6221)

In [24]:
# from torch.utils.data import Dataset

# class CustomImageDataset(Dataset):
#     def __init__(self, source=None, target=None):
#         self.source = source
#         self.target = target

#     def __len__(self):
#         return self.source.shape[0]

#     def __getitem__(self, idx):
#         return self.source[idx], self.target[idx]

# train_iter = CustomImageDataset(train_df["eng_text"], train_df["kor_text"])
# valid_iter = CustomImageDataset(valid_df["eng_text"], valid_df["kor_text"])

In [25]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from konlpy.tag import Hannanum
from konlpy.tag import Okt


def generate_tokens(text_iter, language):
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for text in text_iter:
        yield token_transform[language](text[language_index[language]])


SRC_LANGUAGE = "eng"
TGT_LANGUAGE = "kor"
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3 # 알 수 없음 / 패딩 / 인덱스 시작 / 인덱스 끝 
special_symbols = ["<unk>", "<pad>", "<bos>", "<eos>"]

token_transform = {
    SRC_LANGUAGE: get_tokenizer("spacy", language="en"),
    TGT_LANGUAGE: get_tokenizer(Okt().morphs),
}
print("Token Transform:")
print(token_transform)

vocab_transform = {} # 딕셔너리 {"eng":어쩌구, "kor":저쩌구}
train_iter = train_df.values # 현길사마가 도와준 부분 
val_iter = valid_df.values

for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # train_iter = Multi30k(split="train", language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    vocab_transform[language] = build_vocab_from_iterator(
        generate_tokens(train_iter, language),
        min_freq=1, # 토큰화된 단어들의 최소 빈도수 지정 
        specials=special_symbols, # 트랜스포머에 사용하는 특수토큰 ["<unk>", "<pad>", "<bos>", "<eos>"]
        special_first=True, # 특수 토큰을 단어 집합의 맨 앞에 추가 
    )

for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[language].set_default_index(UNK_IDX)

print("Vocab Transform:")
print(vocab_transform)

c:\Users\kdp\anaconda3\envs\TORCH_NLP38\lib\site-packages\torchtext\data\utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


Token Transform:
{'eng': functools.partial(<function _spacy_tokenize at 0x000002877BE710D0>, spacy=<spacy.lang.en.English object at 0x0000028769472550>), 'kor': <bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x000002876941C5E0>>}
Vocab Transform:
{'eng': Vocab(), 'kor': Vocab()}


In [26]:
print(vocab_transform["eng"].get_itos())  # 단어 리스트 출력
print(vocab_transform["eng"].get_stoi())  # string to index 인덱스-> 단어 / 즉, 단어를 인덱스로 매핑하는 딕셔너리 반환

['<unk>', '<pad>', '<bos>', '<eos>', 'the', 'and', 'of', '.', 'to', 'that', ':', 'in', 'he', ';', 'shall', 'unto', 'for', 'i', 'his', 'a', 'lord', 'they', 'be', 'is', 'not', 'him', 'them', 'it', 'with', 'all', 'thou', 'thy', 'was', 'god', 'which', 'my', 'me', 'said', 'but', 'ye', 'their', 'have', 'will', 'thee', 'from', 'as', '?', 'are', 'when', 'this', 'out', 'were', 'upon', 'man', 'by', 'you', 'israel', 'king', 'son', 'up', 'there', 'hath', 'then', 'people', 'came', 'had', 'house', 'into', 'on', 'her', 'come', 'one', 'we', 'children', 'before', 'your', 'also', 's', 'day', 'land', 'an', 'so', 'men', 'against', 'shalt', 'if', 'at', 'let', 'go', 'hand', 'us', 'saying', 'made', 'went', 'even', 'no', 'do', 'now', 'behold', 'saith', 'therefore', 'every', 'these', 'because', 'after', 'our', 'things', 'down', 'father', 'or', 'sons', 'hast', 'o', 'david', 'say', 'make', 'may', 'over', 'did', 'earth', 'what', 'jesus', 'she', 'who', 'great', 'name', 'thine', 'among', 'away', 'any', 'put', 'ther

In [29]:
import math
import torch
from torch import nn


# 포지셔 
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )

        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# TokenEmbedding 클래스로 소스 데이터와 입력 데이터를 입력 임베딩으로 변환
class Seq2SeqTransformer(nn.Module):
    def __init__(
        self,
        num_encoder_layers,
        num_decoder_layers,
        emb_size,
        max_len,
        nhead,
        src_vocab_size,
        tgt_vocab_size,
        dim_feedforward,
        dropout=0.1,
    ):
        super().__init__()
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            d_model=emb_size, max_len=max_len, dropout=dropout
        )
        # 트랜스포머
        self.transformer = nn.Transformer(
            d_model=emb_size, # 임베딩 차원 : 트랜스포머 모델의 입력과 출력 차원의 크기 정의 
            nhead=nhead, # 멀티 헤드 어텐션의 헤드의 개수 정의 -> 많을수록 병렬처리 능력 증가 
            # 인코더 디코더 개층 수
            num_encoder_layers=num_encoder_layers, 
            num_decoder_layers=num_decoder_layers,
        
            dim_feedforward=dim_feedforward, # 은닉층 크기 정의 
            dropout=dropout,
        )
        self.generator = nn.Linear(emb_size, tgt_vocab_size)

    def forward(
        self,
        # 소스와 타깃 => 인코더 디코더에 대한 시퀀스[시퀀스 길이, 배치 크기, 임베딩 차원]
        src,
        trg,
        # 소스 마스크, 타깃 마스크 => 소스와 타깃의 시퀀스의 마스크
        src_mask,
        tgt_mask,
        src_padding_mask,
        tgt_padding_mask,
        memory_key_padding_mask,
    ):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(
            src=src_emb,
            tgt=tgt_emb,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            memory_mask=None,
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=memory_key_padding_mask
        )
        return self.generator(outs)

    def encode(self, src, src_mask):
        return self.transformer.encoder(
            self.positional_encoding(self.src_tok_emb(src)), src_mask
        )

    def decode(self, tgt, memory, tgt_mask):
        return self.transformer.decoder(
            self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask
        )

In [30]:
from torch import optim
from torchinfo import summary

BATCH_SIZE = 128
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

model = Seq2SeqTransformer(
    num_encoder_layers=3,
    num_decoder_layers=3,
    emb_size=128,
    max_len=128,
    nhead=8,
    src_vocab_size=len(vocab_transform[SRC_LANGUAGE]),
    tgt_vocab_size=len(vocab_transform[TGT_LANGUAGE]),
    dim_feedforward=512,
).to(DEVICE)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)
optimizer = optim.Adam(model.parameters())

summary(model)


cpu


c:\Users\kdp\anaconda3\envs\TORCH_NLP38\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Layer (type:depth-idx)                                                 Param #
Seq2SeqTransformer                                                     --
├─TokenEmbedding: 1-1                                                  --
│    └─Embedding: 2-1                                                  1,601,024
├─TokenEmbedding: 1-2                                                  --
│    └─Embedding: 2-2                                                  3,574,400
├─PositionalEncoding: 1-3                                              --
│    └─Dropout: 2-3                                                    --
├─Transformer: 1-4                                                     --
│    └─TransformerEncoder: 2-4                                         --
│    │    └─ModuleList: 3-1                                            594,816
│    │    └─LayerNorm: 3-2                                             256
│    └─TransformerDecoder: 2-5                                         --
│    │    └─M

In [31]:
# next(iter(valid_iter))

In [32]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence


def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def input_transform(token_ids):
    return torch.cat(
        (torch.tensor([BOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX]))
    )

def collator(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch


text_transform = {}
for language in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[language] = sequential_transforms(
        token_transform[language], vocab_transform[language], input_transform
    )

# data_iter = Multi30k(split="valid", language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collator) # data_iter가 데이터셋인듯
source_tensor, target_tensor = next(iter(dataloader)) 

print("(source, target):")
print(next(iter(val_iter)))

print("source_batch:", source_tensor.shape)
print(source_tensor)

print("target_batch:", target_tensor.shape)
print(target_tensor)

(source, target):
['and they went out quickly and fled from the sepulchre ; for they trembled and were amazed : neither said they any thing to any man ; for they were afraid .'
 '여자들이 심히 놀라 떨며 나와 무덤에서 도망하고 무서워하여 아무에게 아무 말도 하지 못하더라']
source_batch: torch.Size([48, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   5,   97,    5,  ...,    5,    5,  917],
        [  21,   48,  122,  ...,  355, 1220,  274],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
target_batch: torch.Size([47, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [ 251,   79, 1228,  ...,    7, 1305,   50],
        [  11,   31,    9,  ...,  982,    4,  224],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])


In [33]:
def generate_square_subsequent_mask(s):
    mask = (torch.triu(torch.ones((s, s), device=DEVICE)) == 1).transpose(0, 1)
    mask = (
        mask.float()
        .masked_fill(mask == 0, float("-inf"))
        .masked_fill(mask == 1, float(0.0))
    )
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


target_input = target_tensor[:-1, :]
target_out = target_tensor[1:, :] # 어따 쓰노? 

source_mask, target_mask, source_padding_mask, target_padding_mask = create_mask(
    source_tensor, target_input
)

print("source_mask:", source_mask.shape)
print(source_mask)
print()
print("target_mask:", target_mask.shape)
print(target_mask)
print()
print("source_padding_mask:", source_padding_mask.shape)
print(source_padding_mask)
print()
print("target_padding_mask:", target_padding_mask.shape)
print(target_padding_mask)
print()

source_mask: torch.Size([48, 48])
tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])
target_mask: torch.Size([46, 46])
tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]])
source_padding_mask: torch.Size([128, 48])
tensor([[False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ...,  True,  True,  True],
        ...,
        [False, False, False,  ...,  True,  True,  True]

In [34]:
from tqdm import tqdm

def run(model, optimizer, criterion, split):
    model.train() if split == "train" else model.eval()
    data_iter = train_iter if split == 'train' else val_iter
    # data_iter = Multi30k(split=split, language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    dataloader = DataLoader(data_iter, batch_size=BATCH_SIZE, collate_fn=collator)
    
    losses = 0
    pbar = tqdm(dataloader, total=len(list(dataloader)))
    for source_batch, target_batch in pbar:
        source_batch = source_batch.to(DEVICE)
        target_batch = target_batch.to(DEVICE)

        target_input = target_batch[:-1, :]
        target_output = target_batch[1:, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(
            source_batch, target_input
        )

        logits = model(
            src=source_batch,
            trg=target_input,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            src_padding_mask=src_padding_mask,
            tgt_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=src_padding_mask,
        )

        optimizer.zero_grad()
        loss = criterion(logits.reshape(-1, logits.shape[-1]), target_output.reshape(-1))
        if split == "train":
            loss.backward()
            optimizer.step()
            pbar.set_description(f"loss: {loss:.6f}")
        losses += loss.item()

    return losses / len(list(dataloader))

In [35]:
EPOCH = 5
for epoch in range(EPOCH):
    train_loss = run(model, optimizer, criterion, "train")
    val_loss = run(model, optimizer, criterion, "valid")
    print(f"Epoch: {epoch+1}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}")

  0%|          | 0/243 [00:00<?, ?it/s]c:\Users\kdp\anaconda3\envs\TORCH_NLP38\lib\site-packages\torch\nn\functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████| 49/49 [01:44<00:00,  2.13s/it]


Epoch: 1, Train loss: 6.902, Val loss: 5.986


100%|██████████| 49/49 [03:28<00:00,  4.25s/it]


Epoch: 2, Train loss: 5.892, Val loss: 5.404


loss: 5.794723:  70%|██████▉   | 170/243 [24:21<10:27,  8.60s/it]


KeyboardInterrupt: 

In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, 'eng2kor0421.pth')

In [36]:
def greedy_decode(model, source_tensor, source_mask, max_len, start_symbol):
    source_tensor = source_tensor.to(DEVICE)
    source_mask = source_mask.to(DEVICE)

    memory = model.encode(source_tensor, source_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len - 1):
        memory = memory.to(DEVICE)
        target_mask = generate_square_subsequent_mask(ys.size(0))
        target_mask = target_mask.type(torch.bool).to(DEVICE)

        out = model.decode(ys, memory, target_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat(
            [ys, torch.ones(1, 1).type_as(source_tensor.data).fill_(next_word)], dim=0
        )
        if next_word == EOS_IDX:
            break
    return ys


def translate(model, source_sentence):
    model.eval()
    source_tensor = text_transform[SRC_LANGUAGE](source_sentence).view(-1, 1)
    num_tokens = source_tensor.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model, source_tensor, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX
    ).flatten()
    output = vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))[1:-1]
    return " ".join(output)

In [37]:
# 태초에 하나님이 천지를 창조하시니라
output_oov = translate(model, "In the beginning God created the pasta and the pizza.")
output = translate(model, "In the beginning God created the heaven and the earth.")
print(output_oov)
print(output)

그 들 이 그 들 이 그 들 을 위 에 거 하는 것 이 요
그 들 이 그 들 이 그 들 을 위 에 거 하여 그 들 이
